# Llama_tutorial


### 0. Settings

In [ ]:
!pip install -U langchain langgraph langsmith langchain_experimental langchain_core langchain_ollama langchain_prompts

In [1]:
import os
from dotenv import load_dotenv

def langsmith(project_name=None, set_enable=True):
    load_dotenv()

    if set_enable:
        result = os.environ.get("LANGCHAIN_API_KEY")
        if result is None or result.strip() == "":
            print(
                "LangChain API Key가 설정되지 않았습니다."
            )
            return
        os.environ["LANGCHAIN_ENDPOINT"] = (
            "https://api.smith.langchain.com"  # LangSmith API 엔드포인트
        )
        os.environ["LANGCHAIN_TRACING_V2"] = "true"  # true: 활성화
        os.environ["LANGCHAIN_PROJECT"] = project_name  # 프로젝트명
        print(f"LangSmith 추적을 시작합니다.\n[프로젝트명]\n{project_name}")
    else:
        os.environ["LANGCHAIN_TRACING_V2"] = "false"  # false: 비활성화
        print("LangSmith 추적을 하지 않습니다.")


def env_variable(key, value):
    os.environ[key] = value

In [3]:
langsmith("Llama_Tutorial")

LangSmith 추적을 시작합니다.
[프로젝트명]
Llama_Tutorial


In [ ]:
import warnings

# 경고 메시지 무시
warnings.filterwarnings("ignore")

In [ ]:
from langchain_experimental.tools import PythonREPLTool

python_tool = PythonREPLTool()

print(python_tool.invoke("print(100 + 200)"))

300



In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
# from langchain_core.runnables import RunnaleLambda

# import ollama
from langchain_ollama import ChatOllama
from langchain.prompts import PromptTemplate

def print_and_execute(code, debug=True):
    if debug:
        print("CODE:")
        print(code)
    return python_tool.invoke(code)

# prompt = ChatPromptTemplate.from_messages(
#     [
#         (
#             "system",
#             "You are Raymond Hetting, an expert python programmer, well versed in meta-programming and elegant, concise and short but well documented code. You follow the PEP8 style guide. "
#             "Return only the code, no intro, no explanation, no chatty, no markdown, no code block, no nothing. Just the code.",
#         ),
#         ("human", "{input}"),
#     ]
# )

prompt = PromptTemplate(
    template="""You are an assistant for question-answering tasks.
    
    Use the following documents to answer the question.
    
    If you don't know the answer. just say that you don't know.
    
    Use theree senteces maximum and keep the answer concise:
    Question: {question}
    Answer:
    """,
    input_variables={"question"}
)

# response = ollama.chat(
#     model='llama3.2-vision',
#     messages=[{
#         'role': 'user',
#         'content': 'What is in this image?'
#     }]
# )

llm = ChatOllama(
    model = "llama3.1",
    temperature=0
)

chain = prompt | llm | StrOutputParser

chain.invoke("what time is it now?")


TypeError: BaseModel.__init__() takes 1 positional argument but 2 were given

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser

# JSON
llm = ChatOllama(
    model='llama3.1',
    format='json',
    temperature=0
)

prompt = PromptTemplate(
    template="""
    """
)